In [1]:
from binance.client import Client

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler


import gymnasium as gym
from gymnasium import spaces
import gym_anytrading

from stable_baselines3.common.vec_env import DummyVecEnv,VecNormalize,SubprocVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C,DQN,PPO,TD3
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.callbacks import ProgressBarCallback
from stable_baselines3.common.type_aliases import TrainFrequencyUnit



from gym import envs


from gym_anytrading.envs import StocksEnv
from finta import TA
#tf.config.set_visible_devices([],'GPU')

In [3]:
symbol = 'ETHUSDT'
interval = "1h"
start_time = pd.Timestamp.now() - pd.DateOffset(years=3)

df = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

while start_time < pd.Timestamp.now():
    klines = client.futures_klines(symbol=symbol, interval=interval, startTime=int(start_time.timestamp() * 1000))
    if not klines:
        break

    open_prices = [float(kline[1]) for kline in klines]
    high_prices = [float(kline[2]) for kline in klines]
    low_prices = [float(kline[3]) for kline in klines]
    close_prices = [float(kline[4]) for kline in klines]
    volumes = [float(kline[5]) for kline in klines]
    timestamps = [pd.to_datetime(kline[6], unit='ms') for kline in klines]

    df_chunk = pd.DataFrame({'Date': timestamps, 'Open': open_prices, 'High': high_prices, 'Low': low_prices, 'Close': close_prices, 'Volume': volumes})
    df = pd.concat([df, df_chunk], ignore_index=True)

    start_time = timestamps[-1]

final_df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

final_df = final_df.drop_duplicates(subset='Date')
final_df['Date'] = pd.to_datetime(df['Date'])
final_df.set_index('Date', inplace=True)
final_df.sort_index(inplace=True)
final_df = final_df.dropna()
final_df.head(20)
final_df['SMA']=TA.SMA(final_df,21)
final_df['EMA']=TA.EMA(final_df,21)
final_df['MSD']=TA.MSD(final_df,21)
final_df['RSI']=TA.RSI(final_df)
final_df['STOCH']=TA.STOCH(final_df)
final_df['CMO']=TA.CMO(final_df)
final_df['STC']=TA.STC(final_df)
macd_result = TA.MACD(final_df)
final_df['MACD'] = macd_result['MACD']
final_df['OBV']=TA.OBV(final_df)
final_df['WILLIAMS']=TA.WILLIAMS(final_df)


C:\Users\Aiwass\AppData\Local\Temp\ipykernel_14596\4069276566.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_chunk], ignore_index=True)


In [4]:
final_df.dropna(inplace=True)
final_df.iloc[-1]
min_train=final_df['Close'].min()
max_train=final_df['Close'].max()
for column in final_df.columns:
     final_df[column] = (final_df[column] - final_df[column].min()) / (final_df[column].max() - final_df[column].min())
print(final_df.shape)
final_df.head(5)

(26248, 15)


,Open,High,Low,Close,Volume,SMA,EMA,MSD,RSI,STOCH,CMO,STC,MACD,OBV,WILLIAMS
Date,,,,,,,,,,,,,,,
2021-03-01 07:59:59.999,0.132341,0.130139,0.137822,0.135663,0.030004,0.112745,0.116002,0.134179,0.671794,0.859152,0.658545,0.994317,0.658155,0.850224,0.859152
2021-03-01 08:59:59.999,0.135699,0.138497,0.141587,0.142377,0.060322,0.113916,0.117365,0.140763,0.742464,0.925695,0.744055,0.991476,0.662740,0.852443,0.925695
2021-03-01 09:59:59.999,0.142402,0.147739,0.148320,0.152471,0.097087,0.115738,0.119640,0.154116,0.816812,0.951988,0.825659,0.994317,0.673757,0.856010,0.951988
2021-03-01 10:59:59.999,0.152514,0.153000,0.156369,0.157181,0.063885,0.118087,0.122152,0.164017,0.843210,0.939552,0.852841,0.997159,0.685445,0.858359,0.939552
2021-03-01 11:59:59.999,0.157187,0.152871,0.160777,0.157974,0.044881,0.120215,0.124470,0.174142,0.847449,0.962238,0.857203,1.000000,0.694410,0.860012,0.962238


In [15]:

class MyTradingEnv(gym.Env):
    def __init__(self,df,min,max,starting_amount=1000,fee=0.5,window_size:int=56,episode_length:int=100,render_mode=None,verbose=False):
        super(MyTradingEnv,self).__init__()


        self.render_mode=render_mode                                                                        #for rendering
        self.verbose=verbose
        self.min=min
        self.max=max
        self.df=df                                                                                          #entire set of data
        self.starting_amount=starting_amount                                                                #amount of money in wallet in the beginning
        self.window_size=window_size                                                                        #size of window agent will see at 1 time
        self.fee=fee                                                                                        #amount of fee for seeling or buying
        #self.action_space =  spaces.Box(low=0,high=1,shape=(1,))                                           #buy or sell
        self.action_space =  spaces.Discrete(2)                                                             #buy or sell
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.window_size,15),dtype=np.float32)    # vector of 11, for window size of continuous values
        self.episode_length=episode_length                                                                  #amount of windows the agent will see before finishing



        #temporary variables
        self.balance=starting_amount                                                                        #balance which is set to starting amount at the beginning
        self.current_step=0                                                                                 #step that we are currently in
        self.amount_of_episodes=0                                                                               
        self.actions=0                                                                                      #amount of action possesed
        self.direction=2                                                                                    # 0 = LONG | 1 = SHORT
        self.short_entry_price=1e8
        self.net_worth=starting_amount
        #logging
        self.actions_history=[]                                                                             #history of actions taken for plotting
        self.networth_history=[]                                                                            #So i can plot changes of networth                                                    
        self.reward_history=[]                                                                              #history of rewards
        self.cumsumrewards=[]

   
    def step(self, action):
        # Record the previous net worth before taking the action


        current_price = self.df['Close'].iloc[self.current_step-1]*(self.max-self.min)+self.min       # Current closing price

        if self.direction != action:  # If the action requires changing the position
            if action == 1:  # Closing LONG position
                self.balance += self.actions * current_price  - self.fee
                self.actions = 0

                #starting short position immediatelly
                self.short_entry_price = current_price
                self.balance-=self.fee
                self.actions = self.balance / current_price  # 'Borrowed' shares for SHORT
                if self.verbose:
                    print('closing long position:')

            elif action==0:  # Closing SHORT position and calculating profit
                profit = self.actions * (self.short_entry_price - current_price)
                self.balance += profit  - self.fee
                self.actions = 0

                #starting long position
                self.balance-=self.fee
                self.actions = self.balance / current_price
                self.balance -= self.actions * current_price 



            else:                         #first action, idk what to do here
                pass

        

        self.direction = action  # Update the current direction based on action

        # Calculate the new net worth after taking the action
        if self.direction==0:
            self.net_worth = self.balance + self.actions * current_price - self.fee
        elif self.direction==1:
            self.net_worth=self.balance +self.actions * (self.short_entry_price - current_price) - self.fee
         


        reward = self.net_worth - self.starting_amount
        if reward>0:
            reward=1
        else:
            reward=-1

        # Update histories for plotting and analysis
        self.networth_history.append(self.net_worth)
        self.reward_history.append(reward)
        self.actions_history.append(action)

        
        # Update step and episode counters
        self.current_step += 1
        self.amount_of_episodes += 1

        # Determine if the episode should terminate
        truncated = self.current_step >= len(self.df) - 1 or self.amount_of_episodes > self.episode_length
        terminated = self.net_worth < 0

       


        # Generate the observation for the next step
        new_observation = self._get_observation()
        info = {}  # Additional info, if any
        if self.verbose:

            print('balance:',self.balance,'current price:',current_price,'short entry price:',self.short_entry_price,'actions:',self.actions,'net worth:',self.net_worth,'current step:',self.current_step,'amount of episodes',self.amount_of_episodes,'reward:',reward,'action:',action)
        #print('action:',['buy'if action==0 else 'sell'],'step:',self.current_step,'price:',current_price,'actions:',self.actions,'balance:',self.balance,'Networth:',net_worth,'reward:',reward)
        return new_observation, reward, terminated, truncated, info

    def reset(self,**kwargs):
        
        if self.verbose:
            print('reset!')
        self.cumsumrewards.append(np.sum(self.reward_history))

        self.reward_history=[]
        self.actions_history=[]
        info={}



        self.net_worth=self.starting_amount
        self.balance=self.starting_amount                                                                        #balance which is set to starting amount at the beginning
        self.actions=0                                                                                      #amount of action possesed
        self.direction=2                                                                                    # 0 = LONG | 1 = SHORT


        self.current_step = np.random.randint(self.window_size+1, len(self.df)-1)                               #choosing random starting point from the window size to max size of the data frame-size of the episode
        self.amount_of_episodes=0                    
        self.beginning=self.current_step





        self.short_entry_price=1e8                                                        #it doenst make sense
        observation=self._get_observation()
        
        return observation,info
    def _get_observation(self):
        start = self.current_step - self.window_size
        end = self.current_step
        signal_features = self.df.iloc[start:end][[ 'Open', 'High', 'Low', 'Close', 'Volume', 'SMA', 'EMA', 'RSI', 'MSD','STOCH','CMO','STC','MACD','OBV','WILLIAMS']]
        return signal_features


    def render(self, mode='human', close=False):
        if self.verbose:
            print('action history len:',len(self.actions_history))
        if close:
            plt.close()
            return

        plt.figure(figsize=(10, 6))
        plt.plot(self.networth_history, label='Net Worth')
        plt.axhline(y=self.starting_amount, color='r', linestyle='-', label='Starting Amount')

        plt.title('Net Worth over Time')
        plt.xlabel('Step')
        plt.ylabel('Net Worth')
        plt.legend()
        plt.show()


        plt.figure(figsize=(10,6))

        # Ensure we have a valid range to plot
        max_range = min(len(self.df['Close']), self.beginning + len(self.actions_history) + self.window_size)
        price_range = range(self.beginning, max_range)
        price_data=self.df.iloc[self.beginning:max_range]['Close']*(self.max-self.min)+self.min
        # Plot the price data
        plt.plot(price_range, price_data, label='Close Price')

        # Plot the actions on the price data
        for i, action in enumerate(self.actions_history):
            current_step = self.beginning + i+1
            # Ensure we don't go out of bounds
            if current_step < len(self.df['Close']):
                color = 'green' if action == 0 else 'red'
                point=self.df['Close'].iloc[current_step]*(self.max-self.min)+self.min
                plt.scatter(current_step, point, color=color)

        plt.title(str(self.networth_history[-1]))
        plt.xlabel('Step')
        plt.ylabel('Close Price')
        plt.legend()
        plt.show()
            




        plt.figure(figsize=(10, 5))
        plt.plot(self.cumsumrewards, label='Cumulative Rewards')
        plt.xlabel('Time step')
        plt.ylabel('Cumulative Reward')
        plt.title('Cumulative Rewards Over Time')
        plt.legend()
        plt.grid(True)
        plt.show()
    def close(self)->None:
        pass
    @property
    def num_envs(self):                                                                                     #for progress bar
        return 1  # Assuming you are using a single environment
    

In [ ]:
policy_kwargs = dict(
    net_arch=dict(pi=[372,200, 180,150,120,100], vf=[300,200, 180,150,120,100]) 
)

my_env=MyTradingEnv(final_df,min_train,max_train,render_mode="human",verbose=False)
num_envs = 5
env_makers = [lambda: MyTradingEnv(final_df, min_train,max_train, render_mode="human",verbose=False) for _ in range(num_envs)]
env = DummyVecEnv(env_makers)
#env = VecNormalize(env,  norm_reward=True,norm_obs=False)
#model = PPO('MlpPolicy', env, policy_kwargs=policy_kwargs, verbose=0,n_steps=50,batch_size=250)
model = PPO('MlpPolicy', env, verbose=0,n_steps=50,batch_size=250)
#model = A2C('MlpPolicy', env, policy_kwargs=policy_kwargs, verbose=0,gae_lambda=0.9,seed=42)
#model = A2C('MlpPolicy', env,  verbose=0,gae_lambda=0.9,seed=42)
#model = A2C('MlpPolicy', env, verbose=0,seed=42)
model.learn(total_timesteps=5e6, progress_bar=True)
env.render()
